In [44]:
import pandas as pd


In [45]:
df = pd.read_csv("titanic.csv")
df.head()

,PassengerId,Name,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,1,"Braund, Mr. Owen Harris",3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,0
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,female,38.0,1,0,PC 17599,71.2833,C85,C,1
2,3,"Heikkinen, Miss. Laina",3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,1
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,female,35.0,1,0,113803,53.1000,C123,S,1
4,5,"Allen, Mr. William Henry",3,male,35.0,0,0,373450,8.0500,NaN,S,0


In [46]:
df.drop(['PassengerId','Name','SibSp','Parch','Ticket','Cabin','Embarked'],axis='columns',inplace=True)
df.head()

,Pclass,Sex,Age,Fare,Survived
0,3,male,22.0,7.2500,0
1,1,female,38.0,71.2833,1
2,3,female,26.0,7.9250,1
3,1,female,35.0,53.1000,1
4,3,male,35.0,8.0500,0


In [47]:
inputs = df.drop('Survived',axis='columns')
target = df.Survived
inputs

,Pclass,Sex,Age,Fare
0,3,male,22.0,7.2500
1,1,female,38.0,71.2833
2,3,female,26.0,7.9250
3,1,female,35.0,53.1000
4,3,male,35.0,8.0500
...,...,...,...,...
886,2,male,27.0,13.0000
887,1,female,19.0,30.0000
888,3,female,NaN,23.4500
889,1,male,26.0,30.0000


In [48]:
dummies = pd.get_dummies(inputs.Sex)
dummies.head(3)

,female,male
0,0,1
1,1,0
2,1,0


In [49]:
inputs = pd.concat([inputs,dummies],axis='columns')
inputs.head(3)

,Pclass,Sex,Age,Fare,female,male
0,3,male,22.0,7.2500,0,1
1,1,female,38.0,71.2833,1,0
2,3,female,26.0,7.9250,1,0


I am dropping male column as well because of dummy variable trap theory. One column is enough to repressent male vs female



In [50]:
inputs.drop(['Sex','male'],axis='columns',inplace=True)
inputs.head(3)

,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1


In [51]:
inputs.columns[inputs.isna().any()]


Index(['Age'], dtype='object')

In [52]:
inputs.Age[:10]


0    22.0
1    38.0
2    26.0
3    35.0
4    35.0
5     NaN
6    54.0
7     2.0
8    27.0
9    14.0
Name: Age, dtype: float64

In [53]:
inputs.Age = inputs.Age.fillna(inputs.Age.mean())
inputs.head()


,Pclass,Age,Fare,female
0,3,22.0,7.2500,0
1,1,38.0,71.2833,1
2,3,26.0,7.9250,1
3,1,35.0,53.1000,1
4,3,35.0,8.0500,0


In [54]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)


In [55]:
len(X_train)

712

In [56]:
len(X_test)

179

In [57]:
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [58]:
model.fit(X_train,y_train)

GaussianNB()

In [59]:
model.score(X_test,y_test)


0.7653631284916201

In [60]:
X_test[0:10]

,Pclass,Age,Fare,female
221,2,27.0,13.0000,0
312,2,26.0,26.0000,1
866,2,27.0,13.8583,1
494,3,21.0,8.0500,0
549,2,8.0,36.7500,0
482,3,50.0,8.0500,0
339,1,45.0,35.5000,0
377,1,27.0,211.5000,0
394,3,24.0,16.7000,1
884,3,25.0,7.0500,0


In [61]:
y_test[0:10]

221    0
312    0
866    1
494    0
549    1
482    0
339    0
377    0
394    1
884    0
Name: Survived, dtype: int64

In [62]:
model.predict(X_test[0:10])

array([0, 1, 1, 0, 0, 0, 0, 1, 1, 0], dtype=int64)

In [63]:
model.predict_proba(X_test[:10])

array([[9.34610170e-01, 6.53898295e-02],
       [2.73127202e-01, 7.26872798e-01],
       [2.85694280e-01, 7.14305720e-01],
       [9.68904134e-01, 3.10958657e-02],
       [8.86231996e-01, 1.13768004e-01],
       [9.72324808e-01, 2.76751918e-02],
       [7.62945291e-01, 2.37054709e-01],
       [1.36880514e-07, 9.99999863e-01],
       [4.83458817e-01, 5.16541183e-01],
       [9.70588777e-01, 2.94112234e-02]])

Calculate the score using cross validation

In [67]:
from sklearn.model_selection import cross_val_score
cross_val_score(GaussianNB(),X_train, y_train, cv=5)

array([0.75524476, 0.79020979, 0.72535211, 0.77464789, 0.81690141])